In [1]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 274 kB 5.0 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44091 sha256=9e790076bb22031453ae9aa671e3dc045277ea7f30aebd5aecde9c564f2d11e7
  Stored in directory: /home/marcelo/.cache/pip/wheels/cb/b2/f5/ca72754568a333674fd4758f5d5674f88431e58fb9837cc565
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
torch:  1.10 ; cuda:  cu102
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.10/index.html


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
from sklearn.model_selection import KFold
import os, json, cv2, random, sys
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

import argparse

In [5]:
#!python inference_taska.py

In [4]:
#!pip install xmltodict
sys.path.append('Week1')
from utils_week1 import read_annotations
#import bounding_box

#from LossEvalHook import *
#from MyTrainerAugm import *

In [5]:
def parse_annotation(annotations):
    objs = []
    for annot in annotations:

        bbox = [annot[0], annot[1], annot[2], annot[3]]

        obj = {
            "bbox": bbox,
            "bbox_mode": BoxMode.XYXY_ABS,
            "category_id": 0
        }

        objs.append(obj)

    return objs

In [6]:
def create_splits(X, k=4):
    kf = KFold(n_splits=k)
    kf.get_n_splits(X)
    train_folds= []
    val_folds = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        train_folds.append(X_train)
        val_folds.append(X_val)

    return train_folds, val_folds

In [7]:
def load_dataset(type, thing_classes, set_config):

    frames_path = '../data/images'
    gt_path = '../data/ai_challenge_s03_c010-full_annotation.xml'

    gt = read_annotations(gt_path, use_parked=True)

    num_frames = 2141

    frame_list = np.arange(0,2141)

    # Strategy B
    train_val_frames = frame_list[0:int(len(frame_list)*0.25)]

    #Strategy C
    #np.random.seed(42)
    #train_val_frames = np.random.choice(frame_list,int(len(frame_list)*0.25),replace=False)

    train_folds, val_folds = create_splits(train_val_frames, 4)

    train_fold = train_folds[set_config]
    val_fold = val_folds[set_config]

    dataset_dicts = []
    for frame_id in train_val_frames:
        if frame_id in train_fold and type == 'train' or frame_id in val_fold and type != 'train':
            record = {}
            filename = os.path.join(frames_path, "%04d.jpeg" % (frame_id+1))

            record["file_name"] = filename
            # record["image_id"] = i+j
            record["image_id"] = filename
            record["height"] = 1080
            record["width"] = 1920

            record["annotations"] = parse_annotation(gt[frame_id])
            dataset_dicts.append(record)

    return dataset_dicts

In [8]:
#args = parse_args()

model = 'COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'  #pre-trained detectron2 model
print('[INFO] Using model: ', model)

#Hyper-parameters
lr= 0.001     #learning rate
iter= 3000    #max iterations (epochs)
batch= 512    #batch size
set_config=0  #cross val set combination


###-------TRAIN-----------------------------
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file(model))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model)

cfg.OUTPUT_DIR = 'results/task1_3/strategyB/' + 'faster_rcnn' + '/lr_' + str(lr).replace('.', '_') + '_iter_' + str(iter) + '_batch_' + str(batch) + '/' + str(set_config) + '/'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

thing_classes = ['Car']
dataset='AICity'

for d in ['train', 'val', 'test']:
  DatasetCatalog.register(dataset + '_' + d, lambda d=d: load_dataset(d, thing_classes, set_config))
  MetadataCatalog.get(dataset + '_' + d).set(thing_classes=thing_classes)

metadata = MetadataCatalog.get(dataset + '_train')

cfg.DATASETS.TRAIN = (dataset + '_train',)
cfg.DATASETS.VAL = (dataset + '_val',)
cfg.DATASETS.TEST = (dataset + '_test',)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(thing_classes)
cfg.SOLVER.IMS_PER_BATCH = 2

cfg.SOLVER.BASE_LR = lr
cfg.SOLVER.MAX_ITER = iter
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = batch

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# cfg.TEST.EVAL_PERIOD = 0 #eval_period
# trainer = MyTrainer(cfg)
# trainer.resume_or_load(resume=False)
# trainer.train()

###-------INFERENCE AND EVALUATION---------------------------
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained

### MAP #####
#We can also evaluate its performance using AP metric implemented in COCO API.
evaluator = COCOEvaluator(dataset + '_val', cfg, False, output_dir=cfg.OUTPUT_DIR)
test_loader = build_detection_test_loader(cfg, dataset + '_val')
print('------------------------ Evaluating model ' + model + ' on validation set ---------------------------------')
print(inference_on_dataset(trainer.model, test_loader, evaluator))
print('---------------------------------------------------------')




[INFO] Using model:  COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml
[04/05 23:48:49 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBat

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[04/05 23:48:52 d2.engine.train_loop]: Starting training from iteration 0


/home/marcelo/miniconda3/envs/T3-M1/lib/python3.6/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/05 23:49:00 d2.utils.events]:  eta: 0:21:13  iter: 19  total_loss: 1.626  loss_cls: 0.6869  loss_box_reg: 0.8653  loss_rpn_cls: 0.02807  loss_rpn_loc: 0.04837  time: 0.4147  data_time: 0.0112  lr: 1.9981e-05  max_mem: 2644M
[04/05 23:49:09 d2.utils.events]:  eta: 0:21:08  iter: 39  total_loss: 1.497  loss_cls: 0.5713  loss_box_reg: 0.868  loss_rpn_cls: 0.0216  loss_rpn_loc: 0.03448  time: 0.4193  data_time: 0.0033  lr: 3.9961e-05  max_mem: 2644M
[04/05 23:49:17 d2.utils.events]:  eta: 0:20:55  iter: 59  total_loss: 1.388  loss_cls: 0.4652  loss_box_reg: 0.8579  loss_rpn_cls: 0.02494  loss_rpn_loc: 0.03871  time: 0.4156  data_time: 0.0032  lr: 5.9941e-05  max_mem: 2644M
[04/05 23:49:25 d2.utils.events]:  eta: 0:20:47  iter: 79  total_loss: 1.319  loss_cls: 0.3962  loss_box_reg: 0.8585  loss_rpn_cls: 0.0186  loss_rpn_loc: 0.0365  time: 0.4158  data_time: 0.0035  lr: 7.9921e-05  max_mem: 2644M
[04/05 23:49:34 d2.utils.events]:  eta: 0:20:38  iter: 99  total_loss: 1.25  loss_cls: 0.349

KeyboardInterrupt: 